1. Parse https://rusdisinfo.voxukraine.org/

In [363]:
import time
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.safari.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from django.core.validators import URLValidator
from django.core.exceptions import ValidationError

In [364]:
url = "https://rusdisinfo.voxukraine.org/"

In [396]:
def get_links(url):
    options = Options()
    options.add_argument("--headless")

    driver = webdriver.Safari(options=options)
    driver.get(url)

    page = driver.page_source
    soup = BeautifulSoup(''.join(page), 'html.parser')

    titles = soup.find_all('a')
    links = []

    start = "/narratives/"
    for title in titles:
        if title["href"].startswith(start):
            links.append(title["href"])
    
    links_df = pd.DataFrame(columns = ["News", "Fake"])

    count = 0
    for i, link in enumerate(links):
        driver.get(url + link)
        sub_page = driver.page_source
        sub_soup = BeautifulSoup(''.join(sub_page), 'html.parser')

        headers = sub_soup.find_all('h3', class_='Narrative_fakeLink___YbTe')

        
        for j, header in enumerate(headers):
            driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div[2]/div[2]/h3[{j+1}]').click()

            sub_page = driver.page_source
            sub_soup = BeautifulSoup(''.join(sub_page), 'html.parser')
            fakes = sub_soup.find_all('a', target='_blank')

            for fake in fakes:
                if ((links_df[links_df["News"] == fake["href"]].empty)):
                    links_df.loc[count, "News"] = fake["href"]
                    count += 1
            driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div[2]/div[2]/h3[{j+1}]').click()
    
    driver.quit()
    links_df['Fake'] = 1
    
    return links_df.fillna("")

In [397]:
df = get_links(url)

In [398]:
df

,News,Fake
0,https://it.sputniknews.com/20211209/putin-quel...,1
1,https://www.anti-spiegel.ru/2021/krieg-im-donb...,1
2,https://de.rt.com/meinung/129862-lebkuchenhexe...,1
3,https://www.wsws.org/de/articles/2021/04/23/uk...,1
4,https://de.rt.com/meinung/128478-russland-und-...,1
...,...,...
408,https://www.controinformazione.info/gli-accord...,1
409,https://it.sputniknews.com/20220126/Russia-ape...,1
410,https://it.sputniknews.com/20220126/rappresent...,1
411,https://it.sputniknews.com/20200311/al-via-in-...,1


2. Write a function to parse news pages (choose your own media or try to make a general one)

In [399]:
def parse(link):
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Safari(options=options)
    
    
    val = URLValidator()
    try:
        val(f'{link}')
    except:
        driver.quit()
        return ""
    
    driver.execute_script(f"window.location.href = '{link}'")

    time.sleep(7)

    page = driver.page_source
    soup = BeautifulSoup(''.join(page), 'html.parser')

    articles = soup.find_all('p')
    articles_text = []

    for article in articles:
        articles_text.append(article.text)

    driver.quit()
    return articles_text

In [400]:
news = parse("https://www.bbc.com/news")

In [401]:
news

['Rescue teams in both countries are searching for survivors under the rubble after a 7.8-magnitude quake, and subsequent 7.5 tremor, caused hundreds of buildings to collapse.',
 'The death toll continues to rise after two powerful tremors struck as people slept.',
 'Five people are thought to have died as a particularly bloody period between the two sides continues.',
 'A large Russian attack is likely to come from the east and south, Kyiv says.',
 'A Chinese spokeswoman says the aircraft is for civilian use, but "deviated" from its intended route.',
 'The 16-year-old was riding jet skis and swimming with friends when she was attacked, police say.',
 'A British woman tells the BBC how the influencer persuaded her to work for his webcam business.',
 "The novelist was stabbed at a talk in the US last year and says he's found it hard to write since.",
 'An evacuation order is in place within one-mile radius of the scene amid fears of flying shrapnel.',
 'The Premier League charges Manche

In [402]:
# df['News'] = df['News'].apply(parse)